# Tail Recursion

This lecture is about solving the extra credit problem on Project 4. This also brings up a very important concept in programming languages: `Tail Recursion`. 

## Functional Programming

Functional programming is the idea that we can organize an entire program according to pure functions, which are modular and can be combined in an interesting way. 

In functional programming:

1. All functions are pure functions
    * There's no side effect
2. There's no re-assignment and no mutable data types
    * When we create something, it says like that permanently
3. Name-value bindings are permanent

Advantages of functional programming:
1. The value of an expression is independent of the order in which sub-expressions are evaluated
2. Sub-expressions can be safely be valuated in parallel (if we have multiple processing units) or on demand (wait and see if we need the value of a sub-expression before evaluating it, a.k.a. lazily)

The 2 things above are the product of **referential transparency**, the idea that the value of an expression does not change when we substitute one of its subexpression with the value of that subexpression.
* This allows us to do memoization without worrying that we're changing the behavior of our program.

There are many excitements with functional programming, especially nowadays that computers are having more processing units which make it more feasible to parallelize work automatically. 

However, there will be no `for` or `while` statements! Is it still possible to make an efficient iteration process? Is recursion the only way?

Answer: we can make functional programming efficient using tail recursion!

## Recursion and Iteration in Python

In Python, recursive calls always create new active frames.

Let's say we write a function `factorial(n, k)` that computes `k` $\times n!$. The recursion definition is as the following,

In [ ]:
def factorial(n, k):
    if n == 0:
        return k
    else:
        return factorial(n-1, k * n)

The recursive definition takes $\Theta (n)$ (linear) amount of time and $\Theta (n)$ (linear) amount of space.

Meanwhile, the iterative definition is as the following,

In [ ]:
def factorial(n, k):
    while n > 0:
        n, k = n-1, k * n
    return k

The iterative version also takes $\Theta (n)$ (linear) amount of time, but $\Theta (1)$ (constant) space since the program only has 2 names that it needs to remember, `n` and `k`!

How do we bridge the gap between the space efficiency of the recursive version vs. the iterative version?

## Tail Recursion

If we read the specification of the Scheme language from the `Revised Report on the Algorithmic Language Scheme`:

"Implementations of Scheme are required to be **properly tail-recursive**. This allows the execution of an iterative computation in constant space, even if the iterative computation is described by a syntactically recursive procedure."

The idea is that if we write the same logic of the iterative version as a Scheme procedure, 

In [ ]:
(define (factorial n k)
  (if (zero? n) k
      (factorial (- n 1)
                 (* k n)
                 ) ; End of recursive factorial
      ) ; End of if statement
  ) ; End of factorial definition

The Scheme implementation above should use resources like the Python version of the iterative implementation (linear time, constant space)! But how?

**Eliminate the middleman!**

Eliminate frames that we don't need anymore when we make recursive calls. 

## Demo

In [1]:
%load_ext tutormagic

In [2]:
%%tutor --lang python3 -h 500

def factorial(n, k):
    if n == 0:
        return k
    else:
        return factorial(n-1, k * n)

factorial(4, 10)

As we can see above, each recursive call to `factorial` creates a new frame. For each `factorial` call, `n` varies. Eventually, we'll get to the point where we call `factorial` on `n` = 0, and finally we obtain the return value 240.

<img src = 'factorial.png' width = 150>

However, instead of returning the value immediately, Python works its way through the frames that were used.

<img src = 'inefficient.png' width = 130>

Once we obtain the end return value 240, it's unnecessary to keep `n` and `k`. The cascade of frames were needed only to process the recursive calls initially, but they're not needed to return the end return value. Python keeps them around since that's how Python works by default. A proper tail-recursive language gets rid of this behavior.